In [2]:
import pandas as pd
import geopandas as gpd
import numpy as np
import math

## Tsunami in Christchurch Case Study Code

Import important dataframes:  
- *outlines*: New Zealand building outlines
- *cc_affected*: affected SA1 areas in Christchurch

In [ ]:
outlines = gpd.read_file('../QGIS/lds-nz-building-outlines-SHP_2/nz-building-outlines.shp')

In [768]:
cc_affected = gpd.read_file('../NZ_Data/case_study/christchurch/cc_affected.geojson')

<a id='joined_cell'></a>
**joined:** Spatial join the buildings outlines data and the affected SA1 areas in Christchurch  
export joined to csv and geojson as *cc_affected_buildings.csv* and *cc_affected_buildings.geojson*  

In [770]:
joined = gpd.sjoin(cc_affected, outlines, how = 'inner')

In [772]:
joined.to_csv('../NZ_Data/case_study/christchurch/cc_affected_buildings.csv')

In [773]:
joined.to_file('../NZ_Data/case_study/christchurch/cc_affected_buildings.geojson', driver='GeoJSON')

Import the rating units data as *rating_units*

In [48]:
rating_units = gpd.read_file('../NZ_Data/case_study/Rating_Units/Rating_Units.shp')

#### Use Peninsula as smaller scale

I first did a population risk assessment on the peninsula to check that everything worked.

In [10]:
peninsula_sa2 = 331700

In [12]:
peninsula = joined[joined['SA22018_code'] == peninsula_sa2]

In [14]:
# get SA1 districts
sa1 = list(set(peninsula['SA12018_V1']))

Distribute population at building level for day and night

In [17]:
# spilt population between houses equally
# get the number of buildings in each sa1
lst = list(peninsula['SA12018_V1'])
num_house = {}
for l in lst:
    if l in num_house:
        num_house[l] = num_house[l] + 1
    else:
        num_house[l] = 0

In [19]:
per_house_day = {}
per_house_night = {}
for a in sa1:
    row = cc_affected[cc_affected['SA12018_V1'] == a]
    # get the total day and night resident population
    num_day = float(row['C18_CURPop'])
    num_night = float(row['C18_CNPop'])
    per_house_day[a] = num_day / num_house[a]
    per_house_night[a] = num_day / num_house[a]

### Do this with all districts with houses directly affected by tsunami

<a id='most_affected_cell'></a>
Use a list of all SA2 districts to get the SA1 districts affected by the tsunami. Make a dataframe of SA2 districts: *df*. Merge this with the *cc_affected* dataframe to get **most_affected**.  
Export this to csv and geojson:  
- cc_most_affected.csv
- cc_most_affected.geojson

In [776]:
sa2 = [331700, 332400, 332700, 331800, 331500, 331100, 330200, 328300, 327200, 317100, 327500, 326000, 317200, 326200, 324000, 325100, 330700, 332000, 332100]

In [1539]:
df = pd.DataFrame(sa2, columns=['SA2'])
most_affected = pd.merge(cc_affected, df, left_on = 'SA22018_code', right_on='SA2', how='right', validate='m:1')
most_affected = most_affected.drop(['SA2'], axis = 1)

In [780]:
most_affected.to_csv('../NZ_Data/case_study/christchurch/cc_most_affected.csv')

In [782]:
most_affected.to_file('../NZ_Data/case_study/christchurch/cc_most_affected.geojson', driver='GeoJSON')

**affected_buildings**: merge the [joined](#joined_cell) dataframe with the sa1 districts to get the buildings in the affected districts  
Export this to a csv: cc_affected_buildings.csv

In [784]:
affected_buildings = pd.merge(joined, df, left_on = 'SA22018_code', right_on='SA2', how='right', validate='m:1')
affected_buildings = affected_buildings.drop(['SA2'], axis = 1)

In [786]:
affected_buildings.to_csv('../NZ_Data/case_study/christchurch/cc_affected_buildings.csv')

<a id='joined2_cell'></a>**joined2**: spatial join the [most_affected](#most_affected_cell) dataframe with the rating_units data

In [1541]:
joined2 = gpd.sjoin(most_affected, rating_units, how = 'inner')

In [1543]:
joined2 = joined2.reset_index(drop = True)

#### Distribute population at building level for day and night for all affected areas

##### Distributing population accross all buildings

In [490]:
# get the number of buildings in each sa1
# get SA1 districts
num_build = {}
sa1 = list(affected_buildings['SA12018_V1'])
for a in sa1:
    if a in num_build:
        num_build[a] = num_build[a] + 1
    else:
        num_build[a] = 1

In [492]:
per_build_day = {}
per_build_night = {}
for a in sa1:
    row = cc_affected[cc_affected['SA12018_V1'] == a]
    # get total day and night resident population
    num_day = float(row['C18_CURPop'])
    num_night = float(row['C18_CNPop'])
    per_build_day[a] = num_day / num_build[a]
    per_build_night[a] = num_night / num_build[a]

In [1337]:
per_build_day_df = pd.DataFrame.from_dict(per_build_day, orient = 'index', columns = ['day_distribution'])
per_build_day_df = per_build_day_df.reset_index()

In [1339]:
per_build_night_df = pd.DataFrame.from_dict(per_build_night, orient = 'index', columns = ['night_distribution'])
per_build_night_df = per_build_night_df.reset_index()

In [1341]:
build_dist_df = pd.merge(per_build_day_df, per_build_night_df, on = 'index')

In [1343]:
build_dist_df.columns = ['SA1', 'day_distribution', 'night_distribution']

In [1345]:
build_dist_df.to_csv('../NZ_Data/case_study/population_risk_assessment/pop_distribution_by_building.csv')

##### Distributing population among non-vacant residential buildings

In [457]:
# spilt population between houses equally
# get the number of buildings in each sa1
# get SA1 districts
sa1 = list(set(most_affected['SA12018_V1']))
num_house = {}
for i in sa1:
    num_house[i] = 0
for index, row in joined2.iterrows():
    if not pd.isnull(row['LandUse']) and row['LandUse'][0] == '9' and row['LandUse'] != '99':
        sa1_ = row['SA12018_V1']
        num_house[sa1_] = num_house[sa1_] + 1

In [469]:
per_house_day = {}
per_house_night = {}
total_day = 0
total_night = 0
for a in sa1:
    row = cc_affected[cc_affected['SA12018_V1'] == a]
    # get the total day and night resident population
    num_day = float(row['C18_CURPop'])
    num_night = float(row['C18_CNPop'])
    if not np.isnan(num_day):
        total_day = total_day + num_day
    if not np.isnan(num_night):
        total_night = total_night + num_night

    if num_house[a] > 0:
        per_house_day[a] = num_day / num_house[a]
        per_house_night[a] = num_night / num_house[a]
    else:
        per_house_day[a] = 0
        per_house_night[a] = 0

In [471]:
print(f'Number of people impacted if tsunami strikes during the day: {total_day}\nNumber of people impacted if tsunami strikes at night: {total_night}')

Number of people impacted if tsunami strikes during the day: 40950.0
Number of people impacted if tsunami strikes at night: 40893.0


In [1309]:
per_house_day_df = pd.DataFrame.from_dict(per_house_day, orient = 'index', columns = ['day_distribution'])
per_house_day_df = per_house_day_df.reset_index()

In [1311]:
per_house_night_df = pd.DataFrame.from_dict(per_house_night, orient = 'index', columns = ['night_distribution'])
per_house_night_df = per_house_night_df.reset_index()

In [1313]:
house_dist_df = pd.merge(per_house_day_df, per_house_night_df, on = 'index')

In [1319]:
house_dist_df.columns = ['SA1', 'day_distribution', 'night_distribution']

In [1335]:
house_dist_df.to_csv('../NZ_Data/case_study/population_risk_assessment/pop_distribution_by_house.csv')

*Note that 'impact' in this scenario means direct exposure.*

#### Address Points that will be worst affected

Export [joined2](#joined2_cell) to csv as *cc_affected_rating_units.csv*.

In [603]:
joined2.to_csv('../NZ_Data/case_study/christchurch/cc_affected_rating_units.csv')

make a dataframe of the list of affected addresses from the rating units data and export that dataframe as *affected_addresses.csv*

In [52]:
# get the list of affected addresses from the addresses column of the joined2 df
affected_addresses = list(joined2['StreetAddr'])

In [1349]:
affected_addresses_df = pd.DataFrame(affected_addresses, columns = ['address'])

In [1353]:
affected_addresses_df.to_csv('../NZ_Data/case_study/population_risk_assessment/affected_addresses.csv')

#### Households that will require additional support evacuating

To determine the households that will need assistance evacuating we will choose things that could make evacuation difficult and then see which ones each SA1 district needs to worry about most

- Age: households with young children and older adults will have trouble evacuating
    - C18_Age5year_0_4_years
    - C18_Agelifecyclegroup_Over65
- Don't have car: not having a personal car means that people will be relying on public transportation to evacuate
    - C18_MotorVehicles_No_Vehicle
- Disability Status
    - seeing
        - C18_Seeing_A_lot_of_difficulty
        - C18_Seeing_Cannot_do_at_all
    - hearing
        - C18_Hearing_A_lot_of_difficult
        - C18_Hearing_Cannot_do_at_all
    - walking
        - C18_Walking_A_lot_of_difficult
        - C18_Walking_Cannot_do_at_all
    - remembering
        - C18_Remembering_A_lot_of_diffi
        - C18_Remembering_Cannot_do
    - washing
        - C18_Washing_A_lot_of_difficult
        - C18_Washing_Cannot_do_at_all
    - communicating
        - C18_Communicating_A_lot_of_dif
        - C18_Communicating_Cannot_do

In [1500]:
# list of columns I am interested in keeping
col_list = ['C18_Age5year_0_4_years', 'C18_Agelifecyclegroup_Over65', 'C18_MotorVehicles_No_Vehicle', 'C18_Seeing_A_lot_of_difficulty',
            'C18_Seeing_Cannot_do_at_all', 'C18_Hearing_A_lot_of_difficult', 'C18_Hearing_Cannot_do_at_all', 'C18_Walking_A_lot_of_difficult', 'C18_Walking_Cannot_do_at_all',
            'C18_Remembering_A_lot_of_diffi', 'C18_Remembering_Cannot_do', 'C18_Washing_A_lot_of_difficult', 'C18_Washing_Cannot_do_at_all', 'C18_Communicating_A_lot_of_dif',
            'C18_Communicating_Cannot_do', 'SA1', 'SA22018_code']

In [1502]:
assistance = pd.merge(most_affected, df2, left_on = 'SA12018_V1', right_on = 'SA1')

In [1504]:
assistance = assistance[col_list].copy()

The following function will allow us to combine the number of residents who have a lot of difficulty doing something and the number of residents who cannot do it at all.

In [816]:
def combine(diffi, cannot):
    total = []
    for d,c in zip(diffi,cannot):
        if np.isnan(c) and np.isnan(d):
            total.append(np.nan)
        elif np.isnan(d):
            total.append(c)
        elif np.isnan(c):
                total.append(d)
        else:
            total.append(c + d)
    return total

In [1507]:
# combine each set of disability cols into a single col
seeing = combine(list(assistance['C18_Seeing_A_lot_of_difficulty']), list(assistance['C18_Seeing_Cannot_do_at_all']))
hearing = combine(list(assistance['C18_Hearing_A_lot_of_difficult']), list(assistance['C18_Hearing_Cannot_do_at_all']))
walking = combine(list(assistance['C18_Walking_A_lot_of_difficult']), list(assistance['C18_Walking_Cannot_do_at_all']))
remembering = combine(list(assistance['C18_Remembering_A_lot_of_diffi']), list(assistance['C18_Remembering_Cannot_do']))
washing = combine(list(assistance['C18_Washing_A_lot_of_difficult']), list(assistance['C18_Washing_Cannot_do_at_all']))
communicating = combine(list(assistance['C18_Communicating_A_lot_of_dif']), list(assistance['C18_Communicating_Cannot_do']))

Make new columns of these combined totals

In [1509]:
assistance['trouble_seeing'] = seeing
assistance['trouble_hearing'] = hearing
assistance['trouble_walking'] = walking
assistance['trouble_remembering'] = remembering
assistance['trouble_washing'] = washing
assistance['trouble_communicating'] = communicating

Drop the separate columns

In [1513]:
assistance = assistance.drop(['C18_Seeing_A_lot_of_difficulty', 'C18_Seeing_Cannot_do_at_all', 'C18_Hearing_A_lot_of_difficult', 'C18_Hearing_Cannot_do_at_all',
                              'C18_Walking_A_lot_of_difficult', 'C18_Walking_Cannot_do_at_all', 'C18_Remembering_A_lot_of_diffi', 'C18_Remembering_Cannot_do',
                              'C18_Washing_A_lot_of_difficult', 'C18_Washing_Cannot_do_at_all', 'C18_Communicating_A_lot_of_dif', 'C18_Communicating_Cannot_do'], axis=1)

Make dictionaries with the counts of vulnerable populations by SA1. Sort these dictionaries from largest to smallest number of vulnerable residents.

In [1517]:
# get SA1 areas with largest number of young children, older adults, people without a car, and disabled population
young_child = {}
over65 = {}
nocar = {}
seeing = {}
hearing = {}
walking = {}
remembering = {}
washing = {}
communicating = {}
for index, row in assistance.iterrows():
    sa1 = row['SA1']
    o65 = row['C18_Agelifecyclegroup_Over65']
    if not np.isnan(o65):
        over65[sa1] = o65
    young = row['C18_Age5year_0_4_years']
    if not np.isnan(young):
        young_child[sa1] = young
    car = row['C18_MotorVehicles_No_Vehicle']
    if not np.isnan(car):
        nocar[sa1] = car
    see = row['trouble_seeing']
    if not np.isnan(see):
        seeing[sa1] = see
    hear = row['trouble_hearing']
    if not np.isnan(hear):
        hearing[sa1] = hear
    walk = row['trouble_walking']
    if not np.isnan(walk):
        walking[sa1] = walk
    rem = row['trouble_remembering']
    if not np.isnan(rem):
        remembering[sa1] = rem
    wash = row['trouble_washing']
    if not np.isnan(wash):
        washing[sa1] = wash
    comm = row['trouble_communicating']
    if not np.isnan(comm):
        communicating[sa1] = comm

In [1520]:
# put all of these dictionaries in order by value
young_child = {k: v for k, v in sorted(young_child.items(), key=lambda item: item[1], reverse = True)}
over65 = {k: v for k, v in sorted(singleover65.items(), key=lambda item: item[1], reverse = True)}
nocar = {k: v for k, v in sorted(nocar.items(), key=lambda item: item[1], reverse = True)}
disability_count = {k: v for k, v in sorted(disability_count.items(), key=lambda item: item[1], reverse = True)}
seeing = {k: v for k, v in sorted(seeing.items(), key=lambda item: item[1], reverse = True)}
hearing = {k: v for k, v in sorted(hearing.items(), key=lambda item: item[1], reverse = True)}
walking = {k: v for k, v in sorted(walking.items(), key=lambda item: item[1], reverse = True)}
remembering = {k: v for k, v in sorted(remembering.items(), key=lambda item: item[1], reverse = True)}
washing = {k: v for k, v in sorted(washing.items(), key=lambda item: item[1], reverse = True)}
communicating = {k: v for k, v in sorted(communicating.items(), key=lambda item: item[1], reverse = True)}

put each of these dictionaries into a dataframe that can be easily shared and exported as *vulnerable_populations.csv*

In [1361]:
young_child_df = pd.DataFrame.from_dict(young_child, orient = 'index', columns = ['young_child'])
young_child_df = young_child_df.reset_index()

In [1365]:
over65_df = pd.DataFrame.from_dict(over65, orient = 'index', columns = ['over65'])
over65_df = over65_df.reset_index()

In [1367]:
nocar_df = pd.DataFrame.from_dict(nocar, orient = 'index', columns = ['nocar'])
nocar_df = nocar_df.reset_index()

In [1369]:
seeing_df = pd.DataFrame.from_dict(seeing, orient = 'index', columns = ['seeing'])
seeing_df = seeing_df.reset_index()

In [1371]:
hearing_df = pd.DataFrame.from_dict(hearing, orient = 'index', columns = ['hearing'])
hearing_df = hearing_df.reset_index()

In [1373]:
walking_df = pd.DataFrame.from_dict(walking, orient = 'index', columns = ['walking'])
walking_df = walking_df.reset_index()

In [1375]:
remembering_df = pd.DataFrame.from_dict(remembering, orient = 'index', columns = ['remembering'])
remembering_df = remembering_df.reset_index()

In [1377]:
washing_df = pd.DataFrame.from_dict(washing, orient = 'index', columns = ['washing'])
washing_df = washing_df.reset_index()

In [1379]:
communicating_df = pd.DataFrame.from_dict(communicating, orient = 'index', columns = ['communicating'])
communicating_df = communicating_df.reset_index()

In [1381]:
vulnerable_pop_df = young_child_df.merge(over65_df, on = 'index', how = 'outer').merge(nocar_df, on = 'index', how = 'outer').merge(seeing_df, on = 'index', how = 'outer').merge(hearing_df, on = 'index', how = 'outer').merge(walking_df, on = 'index', how = 'outer').merge(remembering_df, on = 'index', how = 'outer').merge(washing_df, on = 'index', how = 'outer').merge(communicating_df, on = 'index', how = 'outer')

In [1389]:
col_names = ['SA1', 'children_0_4_y_count', 'over65_count', 'no_car_count', 'difficulty_seeing_count', 'difficulty_hearing_count', 'difficulty_walking_count', 'difficulty_remembering_count', 'difficulty_washing_count',
             'difficulty_communicating_count']

In [1391]:
vulnerable_pop_df.columns = col_names

In [1395]:
vulnerable_pop_df.to_csv('../NZ_Data/case_study/population_risk_assessment/vulnerable_populations.csv')

SA1 districts with a large number of young children:  
1. 7026534
    - 24
2. 7026558
    - 21
3. 7026580
    - 21
4. 7026391
    - 21
5. 7026563
    - 21

SA1 districts with a large number of residents over 65 years old:  
1. 7026371
    - 105
2. 7026369
    - 84
3. 7026591
    - 78
4. 7026576
    - 75
5. 7026489
    - 69

SA1 districts with a large number of residents who don't have a car:  
1. 7026369
    - 27
2. 7024509
    - 21
3. 7026377
    - 18

SA1 districts with largest number of disabled residents *(residents with multiple disabilities counted multiple times)*:  
1. 7026591
    - 114
2. 7026371
    - 105
3. 7026369
    - 51

SA1 districts with largest number of residents who have difficulty seeing:  
1. 7026591
    - 9
2. 7026371
    - 9
3. 7026451
    - 9
4. 7024539
    - 9

SA1 district with largest number of residents who have difficulty hearing:  
1. 7024568
    - 12
2. 7026362
    - 9
3. 7024577
    - 9

SA1 districts with largest number of residents who have difficulty walking:  
1. 7026591
    - 36
2. 7026369
    - 24
3. 7026371
    - 18
4. 7026362
    - 12
5. 7026531
    - 12
6. 7026395
    - 12
7. 7026470
    - 12

SA1 districts with largest number of residents who have difficulty remembering:  
1. 7026371
    - 27
2. 7026591
    - 21
3. 7026399
    - 12

SA1 districts with largest number of residents who have difficulty washing:  
1. 7026371
    - 33
2. 7026591
    - 27

SA1 districts with largest number of residents who have difficulty communicating:  
1. 7026591
    - 15
2. 7026371
    - 12
3. 7026491
    - 9

#### Impact on GDP

We want to pull the industries that will be most impacted by a tsunami. In this case, that would be industries located in districts that are directly hit.

In [1525]:
cols = most_affected.columns
keep = []
for c in cols:
    if 'Ind22' in c:
        keep.append(c)
keep.append('SA12018_V1')

In [1527]:
most_affected = most_affected[keep].copy()

In [1529]:
# get total number of employees in each industry in the most affected areas
keep.remove('SA12018_V1')
sums = {}
for i in keep:
    lst = list(most_affected[i])
    sums[i] = np.nansum(lst)

In [1531]:
sorted_sums = {k: v for k, v in sorted(sums.items(), key=lambda item: item[1], reverse = True)}

Most Affected:  
1. Manufacturing
2. Construction
3. Health Care and Social Assistance
4. Retail Trade
5. Wholesale Trade
6. Education and Training
7. Transport, Postal, and Warehousing
8. Accommodation and Food Services
9. Professional, Scientific, and Technical Services
10. Administrative and Support Services

Want to figure out what percentage of employees in christchurch will be most affected

**christchurch:** read the christchurch data in as dataframe

In [101]:
christchurch = pd.read_csv('../NZ_Data/case_study/christchurch/cc_data.csv')

In [103]:
# get total number of employees in christchurch
num_emp = 0
for i in keep:
    lst = list(christchurch[i])
    num_emp = num_emp + np.nansum(lst)

In [105]:
# get total number of employees in affected region
num_emp_affected = sum(sums.values())

In [107]:
# get percentage of total employees affected
percent = num_emp_affected / num_emp * 100

In [109]:
# 3 months = 1/4 of a year
yearly_loss_p = percent * 0.25

Losses from affected businesses would account for almost 2% of the GDP of Christchurch if it took them 3 months to recover.

#### Get the important buildings and their locations

The following function determines whether or not a building is important based on it's rating unit land use code.

In [588]:
def is_important(landuse: str) -> bool:
    result = False
    match landuse:
        case '03':
            result = True
        case '06':
            result = True
        case '30':
            result = True
        case '31':
            result = True
        case '33':
            result = True
        case '34':
            result = True
        case '35':
            result = True
        case '42':
            result = True
        case '60':
            result = True
        case '61':
            result = True
        case '62':
            result = True
        case '63':
            result = True
        case '64':
            result = True
        case '65':
            result = True
        case '73':
            result = True
        case '74':
            result = True
        case '75':
            result = True
        case '76':
            result = True
    return result

Make a dictionary of all the important buildings in each SA1 area.

In [590]:
important_places = {}
num_rows = len(joined2.index)
land = list(joined2['LandUse'])
for i in range(0, num_rows):
    landuse = land[i]
    if not pd.isnull(landuse):
        if is_important(landuse):
            sa1_code = joined2['SA12018_V1'][i]
            # print([landuse])
            if sa1_code in important_places:
                # print(sa1_code)
                # print(important_places[sa1_code])
                important_places[sa1_code].append(landuse)
                # print(important_places[sa1_code])
            else:
                # print([landuse])
                important_places[sa1_code] = [landuse]

Get the important places on the peninsula, we want to relocate some of these places, but no all of them because people still live there and need essential services.

In [594]:
# get the SA1 districts in the peninsula
penin = list(joined2[joined2['SA22018_code'] == 331700]['SA12018_V1'])

In [597]:
important_peninsula = {}
for sa1 in important_places:
    if sa1 in penin:
        important_peninsula[sa1] = important_places[sa1]

In [599]:
important_peninsula

{7026572: ['62', '65'],
 7026534: ['65', '62'],
 7026548: ['62'],
 7026551: ['62'],
 7026554: ['64'],
 7026555: ['62'],
 7026556: ['62'],
 7026558: ['62'],
 7026560: ['62', '62', '62'],
 7026573: ['62'],
 7026574: ['62']}

Relocate two electricity utilities buildings in area 7026560 where there are three. This area is very close to the water and would be very affected by a tsunami.

#### Get important places in areas that will be flooded and decide how much should be relocated.

##### SA1: 7026439  
![alternate image name](7026439.png)

In [604]:
counts = {}
for code in important_places[7026439]:
    if code in counts:
        counts[code] = 1 + counts[code]
    else:
        counts[code] = 1

In [606]:
counts

{'65': 2, '35': 1, '76': 1, '64': 1, '62': 4, '74': 2, '73': 2, '75': 6}

Electricity: (62)  
relocate 2 of the 4 electricity buildings further inland  
Food, drink, tobacco: (71)  
relocate 7 of the 13 food, drink, tobacco industrial buildings further inland. Especially food buildings.  
Everything else is less necessary to recovery and don't need to be relocated.

##### Places that don't have buildings to be relocated  
- 7024286
- 7024287
- 7024288
- 7026379
- 7026382
- 7026438
- 7026539
- 7026561
- 7026562
- 7026564
- 7026568
- 7026569
- 7026570
- 7026579
- 7026580
- 7026582
- 7026583
- 7026584
- 7026585
- 7026586
- 7026587
- 7026588
- 7026589
- 7026590
- 7026592
- 7026593
- 7026594
- 7026595

##### Get SA1 districts along the east coast by SA2 district.

In [608]:
sa2_east = [330200, 328300, 327200, 326000]
east_df = pd.DataFrame(sa2_east, columns = ['SA2'])
east_df = pd.merge(christchurch, east_df, left_on='SA22018_code', right_on='SA2', how='right', validate='m:1')
sa1_east = east_df['SA12018_V1_00']

In [610]:
important_east = {}
for sa1 in important_places:
    if sa1 in sa1_east:
        important_east[sa1] = important_places[sa1]

In [612]:
east_lst = {}
for sa1 in sa1_east:
    counts = {}
    if sa1 in important_places:
        for code in important_places[sa1]:
            if code in counts:
                counts[code] = 1 + counts[code]
            else:
                counts[code] = 1
        east_lst[sa1] = counts

Relocate 2 of the electricity facilities (62) in 7026517.  
![alternate image name](7026517.png)

##### SA1: 7026464  
![alternate image name](7026464.png)

In [616]:
counts = {}
for code in important_places[7026464]:
    if code in counts:
        counts[code] = 1 + counts[code]
    else:
        counts[code] = 1

In [618]:
counts

{'62': 8, '65': 2}

Relocate 7 electricity facilities (62) into less affected areas. Move one of the sanitary facilities as well.

##### SA1: 7026563  
![alternate image name](7026563.png)

In [622]:
counts = {}
for code in important_places[7026563]:
    if code in counts:
        counts[code] = 1 + counts[code]
    else:
        counts[code] = 1

In [624]:
counts

{'62': 2, '42': 3}

relocate 1-2 healthcare facilities (42) to the north-western side of the area where the tsunami won't reach or to the area 7026536 to the west.

##### SA1: 7026591  
![alternate image name](7026591.png)

In [628]:
counts = {}
for code in important_places[7026591]:
    if code in counts:
        counts[code] = 1 + counts[code]
    else:
        counts[code] = 1

In [630]:
counts

{'42': 5, '62': 1}

#### Determine buildings that can be turned into vertical evacuation shelters

##### Parking Garages

Find all units labeled as 'parking' (32)

In [635]:
# get the dataframe with only rows of parking facilites
parking_df = joined2[joined2['LandUse'] == '85']
parking_df = parking_df.reset_index(drop = True)

In [637]:
# Get list of addresses of these facilities so that I can check that these facilities are multiple level parking garages
addresses = list(parking_df['StreetAddr'])

None of the Transport: parking (32) facilities are multistory parking garages.  
None of the Commercial: car parking (85) facilities are multistory parking garages.  

#### Peninsula Shelter costs  
![alternate image name](peninsula_shelter.png)

Shelters are numbered 1-5 starting from the southernmost one. I will determine how many people each shelter needs to accomodate based on the populations of each SA1 district it is supporting.

##### **Shelter 1**  
*supporting:* 7026575, 7026574, 7026573, half of 7026572

In [218]:
shelter1 = [7026575, 7026574, 7026573, 7026572]
shelter1_sa1 = pd.DataFrame(shelter1, columns = ['SA1'])

In [220]:
shelter1_sa1 = shelter1_sa1.merge(cc_affected, left_on = 'SA1', right_on = 'SA12018_V1')

In [222]:
pops1 = list(shelter1_sa1['C18_CURPop'])
pops1[3] = pops1[3] / 2

In [224]:
shelter1_pop = sum(pops1)

In [226]:
print(f'Shelter 1 needs to support about {shelter1_pop} people.')

Shelter 1 needs to support about 601.5 people.


##### **Shelter 2**  
*supporting:* half of 7026572, 7026559, 7026560, 7026558

In [229]:
shelter2 = [7026572, 7026559, 7026560, 7026558]
shelter2_sa1 = pd.DataFrame(shelter2, columns = ['SA1'])

In [231]:
shelter2_sa1 = shelter2_sa1.merge(cc_affected, left_on = 'SA1', right_on = 'SA12018_V1')

In [233]:
pops2 = list(shelter1_sa1['C18_CURPop'])
pops2[3] = pops2[0] / 2

In [235]:
shelter2_pop = sum(pops2)

In [237]:
print(f'Shelter 2 needs to support about {shelter2_pop} people.')

Shelter 2 needs to support about 595.5 people.


##### **Shelter 3**  
*supporting:* 7026556, 7026555, 7026554, 7026557

In [240]:
shelter3 = [7026556, 7026555, 7026554, 7026557]
shelter3_sa1 = pd.DataFrame(shelter3, columns = ['SA1'])

In [242]:
shelter3_sa1 = shelter3_sa1.merge(cc_affected, left_on = 'SA1', right_on = 'SA12018_V1')

In [244]:
pops3 = list(shelter3_sa1['C18_CURPop'])

In [245]:
shelter3_pop = sum(pops3)

In [248]:
print(f'Shelter 3 needs to support about {shelter3_pop} people.')

Shelter 3 needs to support about 657.0 people.


##### Shelter 4  
*supporting:* 7026553, 7026552, 7026550, 7026551

In [251]:
shelter4 = [7026553, 7026552, 7026550, 7026551]
shelter4_sa1 = pd.DataFrame(shelter4, columns = ['SA1'])

In [253]:
shelter4_sa1 = shelter4_sa1.merge(cc_affected, left_on = 'SA1', right_on = 'SA12018_V1')

In [255]:
pops4 = list(shelter4_sa1['C18_CURPop'])
shelter4_pop = sum(pops4)

In [257]:
print(f'Shelter 4 needs to support about {shelter4_pop} people.')

Shelter 4 needs to support about 516.0 people.


##### Shelter 5  
*supporting:* 7026549, 7026546, 7026548, 7026531

In [260]:
shelter5 = [7026549, 7026546, 7026548, 7026531]
shelter5_sa1 = pd.DataFrame(shelter5, columns = ['SA1'])

In [262]:
shelter5_sa1 = shelter5_sa1.merge(cc_affected, left_on = 'SA1', right_on = 'SA12018_V1')

In [264]:
pops5 = list(shelter5_sa1['C18_CURPop'])
shelter5_pop = sum(pops5)

In [266]:
print(f'Shelter 5 needs to support about {shelter5_pop} people.')

Shelter 5 needs to support about 510.0 people.


##### Shelter 6  
*supporting:* 7026534, 7026547, 7026533, 7026532

In [269]:
shelter6 = [7026534, 7026547, 7026533, 7026532]
shelter6_sa1 = pd.DataFrame(shelter6, columns = ['SA1'])

In [271]:
shelter6_sa1 = shelter6_sa1.merge(cc_affected, left_on = 'SA1', right_on = 'SA12018_V1')

In [273]:
pops6 = list(shelter6_sa1['C18_CURPop'])
shelter6_pop = sum(pops6)

In [275]:
print(f'Shelter 6 needs to support about {shelter6_pop} people.')

Shelter 6 needs to support about 684.0 people.


##### Shelter 7  
*supporting:* 7026545, 7026530, 7026529, 7026528, 7026527

In [278]:
shelter7 = [7026545, 7026530, 7026529, 7026528, 7026527]
shelter7_sa1 = pd.DataFrame(shelter7, columns = ['SA1'])

In [280]:
shelter7_sa1 = shelter7_sa1.merge(cc_affected, left_on = 'SA1', right_on = 'SA12018_V1')

In [282]:
pops7 = list(shelter7_sa1['C18_CURPop'])
shelter7_pop = sum(pops7)

In [284]:
print(f'Shelter 7 needs to support about {shelter7_pop} people.')

Shelter 7 needs to support about 756.0 people.


##### Shelter 8  
*supporting:* 7026525, 7026524, 7026526, 7026523, 7026520, 7026522

In [287]:
shelter8 = [7026525, 7026524, 7026526, 7026523, 7026520, 7026522]
shelter8_sa1 = pd.DataFrame(shelter8, columns = ['SA1'])

In [289]:
shelter8_sa1 = shelter8_sa1.merge(cc_affected, left_on = 'SA1', right_on = 'SA12018_V1')

In [291]:
pops8 = list(shelter8_sa1['C18_CURPop'])
shelter8_pop = sum(pops8)

In [293]:
print(f'Shelter 8 needs to support about {shelter8_pop} people.')

Shelter 8 needs to support about 888.0 people.


##### Calculating the volume of soil berms

In [296]:
floor = 700 * 20 / .85

In [1205]:
floor_r

72.40684405880809

In [298]:
floor_r = math.sqrt(floor / math.pi)

In [300]:
height = 73.5

In [1139]:
base_r = 4 * height + floor_r

In [1141]:
base = (base_r)**2 * math.pi

In [1207]:
base_r

366.4068440588081

In [1143]:
volume = (1/3) * math.pi * height * (floor_r**2 + floor_r * base_r + base_r**2)

In [1203]:
volume

12778942.386696842

In [1145]:
price = (volume/height/3)/5 * ((19 + 204) / 2)

In [1147]:
price

1292382.835479998

In [1149]:
price_nzd = 2155996

In [1173]:
shelter_cost = price_nzd * 8

It will cost about 879,000 USD to build a single berm. That is about 1,477,401 NZD per berm.

This means that 8 berms will cost about $11,819,208.

##### Relocation Pricing

I am assuming that the price of relocation is the same as the capital value

In [642]:
sa1_relocate = [7026560, 7026517, 7026464, 7026563, 7026591]
sa1_relocate_df = pd.DataFrame(sa1_relocate, columns = ['SA1'])
relocate_buildings = pd.merge(joined2, sa1_relocate_df, left_on='SA12018_V1', right_on='SA1', how='right', validate='m:1')

In [644]:
cap_val = {}
codes = ['62', '65', '42']
for index, row in relocate_buildings.iterrows():
    sa1 = row['SA12018_V1']
    use_code = row['LandUse']
    if use_code in codes:
        if sa1 in cap_val:
            if use_code in cap_val[sa1]:
                cap_val[sa1][use_code].append(row['CapitalVal'])
            else:
                cap_val[sa1][use_code] = [row['CapitalVal']]
        else:
            cap_val[sa1] = {}
            cap_val[sa1][use_code] = [row['CapitalVal']]

for sa1 in cap_val:
    for code in cap_val[sa1]:
        cap_val[sa1][code] = sorted(cap_val[sa1][code])

In [646]:
cap_val

{7026560: {'62': [5500.0, 5500.0, 5500.0]},
 7026517: {'62': [10000.0, 10000.0, 165000.0]},
 7026464: {'62': [200.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, 990000.0],
  '65': [66500.0, 90000.0]},
 7026563: {'62': [7500.0, 7500.0], '42': [570000.0, 820000.0, 1710000.0]},
 7026591: {'42': [165000.0, 165000.0, 180000.0, 180000.0, 7610000.0],
  '62': [130000.0]}}

In [648]:
# get the total cost of moving all buildings
relocation_cost = 0
for sa1 in cap_val:
    for code in cap_val[sa1]:
        for i in range(0, len(cap_val[sa1]) - 1):
            relocation_cost = relocation_cost + cap_val[sa1][code][i]

In [650]:
print(f'It will take about ${relocation_cost} to relocate the important buildings')

It will take about $939200.0 to relocate the important buildings


Raise all floorings in important buildings in most most affected areas. To do this calculate sum of 1/4 of capital values and see how much it costs.

In [1399]:
sa1_raise_floors = [7026568, 7026570, 7026569, 7026582, 7026583, 7026584, 7026585, 7026586, 7026587, 7026588, 7026589, 7026591, 7026590, 7026592, 7026593, 7026594, 7026595, 7026579,
                    7026580, 7026561, 7026564, 7026563]
sa2_raise_floors = [331700, 330200]
# get the SA1 numbers for the SA2 districts
sa2_floors_df = pd.DataFrame(sa2_raise_floors, columns = ['SA2'])
get_sa1 = pd.merge(cc_affected, sa2_floors_df, left_on = 'SA22018_code', right_on = 'SA2', validate = 'm:1')
sa1_sa2_raise_floors = sa1_raise_floors + list(get_sa1['SA12018_V1'])

In [1413]:
sa1_floors_df = pd.DataFrame(sa1_raise_floors, columns = ['SA1'])
sa1_sa2_floors_df = pd.DataFrame(sa1_sa2_raise_floors, columns = ['SA1'])
sa1_raise_floors_df = pd.merge(joined2, sa1_floors_df, left_on = 'SA12018_V1', right_on = 'SA1')
sa1_sa2_raise_floors_df = pd.merge(joined2, sa1_sa2_floors_df, left_on = 'SA12018_V1', right_on = 'SA1')

In [1405]:
raise_floor_cost = 0
capital_vals = []
for index, row in sa1_raise_floors_df.iterrows():
    landuse = row['LandUse']
    if not pd.isnull(landuse):
        if is_important(landuse):
            val = row['CapitalVal']
            raise_floor_cost = raise_floor_cost + (val * 0.25)
            capital_vals.append(val * 0.25)

In [1415]:
raise_floor_cost = 0
capital_vals = []
for index, row in sa1_sa2_raise_floors_df.iterrows():
    landuse = row['LandUse']
    if not pd.isnull(landuse):
        if is_important(landuse):
            val = row['CapitalVal']
            raise_floor_cost = raise_floor_cost + (val * 0.25)
            capital_vals.append(val * 0.25)

Get map of places where important buildings are adapted

In [1428]:
adapted_buildings_df = cc_affected.merge(sa1_sa2_floors_df, left_on = 'SA12018_V1', right_on = 'SA1')

In [1430]:
adapted_buildings_df.to_file('../NZ_Data/case_study/christchurch/adapted_buildings.geojson', driver = 'GeoJSON')

In [1417]:
capital_vals = sorted(capital_vals, reverse = True)

In [1419]:
raise_floor_cost

4119412.5

In [343]:
total_plan1 = raise_floor_cost + relocation_cost + shelter_cost

##### Vulnerable Population

Determine how much it will cost to raise the floors on houses in SA1 districts with vulnerable populations.

In [422]:
sa1_disability = [7026591, 7026371, 7026369]
sa1_disability_df = pd.DataFrame(sa1_disability, columns = ['SA1'])
sa1_disability_floors_df = pd.merge(joined2, sa1_disability_df, left_on = 'SA12018_V1', right_on = 'SA1')

In [424]:
sa1_disability_floors = 0
capital_vals = []
for index, row in sa1_disability_floors_df.iterrows():
    landuse = row['LandUse']
    if not pd.isnull(landuse):
        if landuse[0] == '9':
            val = row['CapitalVal']
            sa1_disability_floors = sa1_disability_floors + (val * 0.25)
            capital_vals.append(val * 0.25)

Figure out how many more vertical evacuation shelters we can build.

In [352]:
shelters_left = (30000000 - total_plan1) / price_nzd

In [354]:
print(f'We can build {int(shelters_left)} more shelters.')

We can build 8 more shelters.


#### Shelters on the Southern Coast

Shelters are labeled from south to north.

##### Shelter 9   
*supporting:* 7026580, 7026579, 7026595, 7016594

In [373]:
shelter9 = [7026580, 7026579, 7026595, 7016594]
shelter9_sa1 = pd.DataFrame(shelter9, columns = ['SA1'])

In [375]:
shelter9_sa1 = shelter9_sa1.merge(cc_affected, left_on = 'SA1', right_on = 'SA12018_V1')

In [377]:
pops9 = list(shelter9_sa1['C18_CURPop'])
shelter9_pop = sum(pops9)

In [379]:
print(f'Shelter 9 needs to support about {shelter9_pop} people.')

Shelter 9 needs to support about 714.0 people.


##### Shelter 10  
*supporting:* 7026593, 7026592, 7026588, 7026589, 7026587, 7026590

In [384]:
shelter10 = [7026593, 7026592, 7026588, 7026589, 7026587, 7026590]
shelter10_sa1 = pd.DataFrame(shelter10, columns = ['SA1'])

In [386]:
shelter10_sa1 = shelter10_sa1.merge(cc_affected, left_on = 'SA1', right_on = 'SA12018_V1')

In [388]:
pops10 = list(shelter10_sa1['C18_CURPop'])
shelter10_pop = sum(pops10)

In [390]:
print(f'Shelter 10 needs to support about {shelter10_pop} people.')

Shelter 10 needs to support about 789.0 people.


##### Shelter 11  
*supporting:* 7026586, 7026585, 7026584, 7026582, 7026583

In [394]:
shelter11 = [7026586, 7026585, 7026584, 7026582, 7026583]
shelter11_sa1 = pd.DataFrame(shelter11, columns = ['SA1'])

In [396]:
shelter11_sa1 = shelter11_sa1.merge(cc_affected, left_on = 'SA1', right_on = 'SA12018_V1')

In [398]:
pops11 = list(shelter11_sa1['C18_CURPop'])
shelter11_pop = sum(pops11)

In [400]:
print(f'Shelter 11 needs to support about {shelter11_pop} people.')

Shelter 11 needs to support about 765.0 people.


In [1189]:
south_shelter_cost = price_nzd * 3

Add 3 more structures on the east coast

In [1171]:
east_shelter_cost = price_nzd * 3

In [1191]:
plan1_total_cost = shelter_cost + south_shelter_cost + raise_floor_cost + relocation_cost + piles_cost

In [1199]:
price_nzd * 11

23715956

In [1193]:
plan1_total_cost

29588828.321788788

In [862]:
relocation_cost

939200.0

##### Reinforce Important buildings in most affected areas by giving them concrete pile foundation

The price of this will be calculated based on size. Take the average size and cost: a 190 sq meter building's piles cost 12,500 nzd. This means that piles cost about 65.79 nzd per sq meter.

In [1421]:
piles_cost = 0
for index, row in sa1_sa2_raise_floors_df.iterrows():
    landuse = row['LandUse']
    if not pd.isnull(landuse):
        if is_important(landuse):
            val = row['AREA_M2'] * 65.79
            piles_cost = piles_cost + val

In [1209]:
price_nzd * 11

23715956

#### Plan 2  
This plan will focus more on reinforcing houses in areas with large vulnerable populations.

First I want to use the vulnerable population data that I found before to determine which areas have the most vulnerable people.  
I'm going to do this 2 ways: see which areas are higher in each area relatively, and which areas have the highest total count.

In [869]:
vulnerable = [young_child, over65, nocar, seeing, hearing, walking, remembering, washing, communicating]

In [871]:
# get areas with highest total count of vulnerable populations
vulnerable_total = {}
for v in vulnerable:
    for a in v:
        if a in vulnerable_total:
            vulnerable_total[a] = vulnerable_total[a] + v[a]
        else:
            vulnerable_total[a] = v[a]

In [875]:
vulnerable_total = {k: v for k, v in sorted(vulnerable_total.items(), key=lambda item: item[1], reverse = True)}

##### Build Vertical Evacuation Shelters in Affected Areas with a large amount of vulnerable people

In [1081]:
vulnerable_sa1 = list(vulnerable_total.keys())

**most_affected_vulnerable**: merge a dataframe of all most affected SA1 areas with vulnerable residents with the [most_affected](#most_affected_cell) dataframe

In [1083]:
vulnerable_sa1_df = pd.DataFrame(vulnerable_sa1, columns = ['SA1'])
most_affected_vulnerable = pd.merge(most_affected, vulnerable_sa1_df, left_on = 'SA12018_V1', right_on = 'SA1')

In [1085]:
df_temp = pd.DataFrame.from_dict(vulnerable_total, orient = 'index', columns = ['vulnerable'])

In [1086]:
df_temp = df_temp.reset_index()

In [1089]:
df_temp.columns = ['SA1', 'vulnerable_count']

In [1091]:
most_affected_vulnerable = pd.merge(most_affected_vulnerable, df_temp, on = 'SA1')

In [1095]:
most_affected_vulnerable.to_file('../NZ_Data/case_study/christchurch/vulnerable_populations.geojson', driver = 'GeoJSON')

9 Shelters

In [1161]:
plan2_shelters = price_nzd * 9

In [1163]:
plan2_shelters

19403964

Use remaining 10 million to add piles to houses in vulnerable areas.

In [1288]:
plan2_cost = sa1_low_income_piles + plan2_shelters

In [1290]:
plan2_cost

29934901.41959841

Want to adapt the homes of people who would have difficulty recovering: People who have a low income.

In [1216]:
low_income = []
for index, row in most_affected.iterrows():
    _5 = 0 if np.isnan(row['C18_GroupedPersIncome_5K_Less']) else row['C18_GroupedPersIncome_5K_Less']
    _10 = 0 if np.isnan(row['C18_GroupedPersIncome_5K_10K']) else row['C18_GroupedPersIncome_5K_10K']
    _20 = 0 if np.isnan(row['C18_GroupedPersIncome_10K_20K']) else row['C18_GroupedPersIncome_10K_20K']
    _30 = 0 if np.isnan(row['C18_GroupedPersIncome_20K_30K']) else row['C18_GroupedPersIncome_20K_30K']

    low_income.append(_5 + _10 + _20 + _30)

In [1218]:
most_affected['low_income'] = low_income

In [1224]:
low_income_dict = {}
for index, row in most_affected.iterrows():
    low_income_dict[row['SA12018_V1']] = row['low_income']

In [1226]:
low_income_dict = {k: v for k, v in sorted(low_income_dict.items(), key=lambda item: item[1], reverse = True)}

In [1230]:
most_affected.to_file('../NZ_Data/case_study/christchurch/low_income.geojson', driver = 'GeoJSON')

In [1276]:
sa1_low_income_piles = [7026371, 7026518, 7026516, 7026379]
sa1_low_income_piles_df = pd.DataFrame(sa1_low_income_piles, columns = ['SA1'])
sa1_low_income_piles_df = pd.merge(joined2, sa1_low_income_piles_df, left_on = 'SA12018_V1', right_on = 'SA1')

In [1278]:
# get the cost of adding concrete pile foundations to houses in most affected areas with large amount of vulnerable population
sa1_low_income_piles = 0
for index, row in sa1_low_income_piles_df.iterrows():
    landuse = row['LandUse']
    if not pd.isnull(landuse):
        if landuse[0] == '9' and landuse != '99':
            val = row['AREA_M2'] * 65.79
            sa1_low_income_piles = sa1_low_income_piles + val

In [1284]:
sa1_low_income_piles

10530937.419598412

In [1434]:
sa1_low_income_adapt = [7026371, 7026518, 7026516, 7026379]
low_income_adapt_df = pd.DataFrame(sa1_low_income_adapt, columns = ['SA1'])

In [1440]:
low_income_adapt_df = most_affected.merge(low_income_adapt_df, right_on = 'SA1', left_on = 'SA12018_V1')

In [1444]:
low_income_adapt_df.to_file('../NZ_Data/case_study/christchurch/low_income_adapt.geojson', driver = 'GeoJSON')